# SSBIO: Collection of functions used throughout a structural reconstruction

***
## Imports

In [60]:
# ESSENTIAL
import os
import ast
import json
import pickle
import operator
import warnings
import numpy as np
import pandas as pd
import collections
from collections import defaultdict
from dateutil.parser import parse as dateparse

In [8]:
# CLEARING OUTPUT
import sys 
import time
from IPython.display import display, clear_output

In [9]:
# SUPPRESSING ALL OUTPUT
from contextlib import contextmanager
import sys, os
 
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [10]:
import sys, time
import itertools
try:
    from IPython.display import clear_output
    have_ipython = True
except ImportError:
    have_ipython = False

class ProgressBar:
    def __init__(self, iterations):
        self.iterations = iterations
        self.prog_bar = '[]'
        self.fill_char = '*'
        self.width = 40
        self.__update_amount(0)
        if have_ipython:
            self.animate = self.animate_ipython
        else:
            self.animate = self.animate_noipython

    def animate_ipython(self, iter):
        print '\r', self,
        sys.stdout.flush()
        self.update_iteration(iter + 1)

    def update_iteration(self, elapsed_iter):
        self.__update_amount((elapsed_iter / float(self.iterations)) * 100.0)
        self.prog_bar += '  %d of %s complete' % (elapsed_iter, self.iterations)

    def __update_amount(self, new_amount):
        percent_done = int(round((new_amount / 100.0) * 100.0))
        all_full = self.width - 2
        num_hashes = int(round((percent_done / 100.0) * all_full))
        self.prog_bar = '[' + self.fill_char * num_hashes + ' ' * (all_full - num_hashes) + ']'
        pct_place = (len(self.prog_bar) // 2) - len(str(percent_done))
        pct_string = '%d%%' % percent_done
        self.prog_bar = self.prog_bar[0:pct_place] + \
            (pct_string + self.prog_bar[pct_place + len(pct_string):])

    def __str__(self):
        return str(self.prog_bar)

In [11]:
# COBRAPY
# used to read GEMs into python
from cobra import Model, Reaction, Metabolite
from cobra.io.sbml import create_cobra_model_from_sbml_file
from cobra.io.mat import load_matlab_model

# LIBSBML
# used to directly read SBML files
from libsbml import *
reader = SBMLReader()

# REQUESTS
# used for interfacing with REST
import requests
import urllib2
import gzip

# STRINGIO
import StringIO as sio

# BIOSERVICES
# http://bioinformatics.oxfordjournals.org/content/29/24/3241
from bioservices.uniprot import UniProt
bsup = UniProt()

# PRODY
# http://bioinformatics.oxfordjournals.org/content/27/11/1575
# used for protein structure information
import prody as pr
pr.confProDy(verbosity='none')

# BIOPYTHON
# used for sequence alignment
from Bio import SeqIO
from Bio import AlignIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
from Bio.PDB.Polypeptide import *
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB import PDBIO
from Bio import PDB

# BEAUTIFULSOUP
from bs4 import BeautifulSoup

# TEMPFILE - for storing files temporarily (cross-platform!)
import tempfile

import re

cobra/solvers/__init__.py:63 UserWarning: No LP solvers found
@> ProDy is configured: verbosity='none'
INFO:.prody:ProDy is configured: verbosity='none'


In [12]:
# PANDAS DISPLAY OPTIONS
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

In [13]:
# MATPLOTLIB FIGURES
import seaborn as sns
sns.set_style("dark")
sns.set_context("poster")

from matplotlib import rcParams
rcParams['figure.dpi'] = 300
rcParams['lines.linewidth'] = 2
rcParams['axes.facecolor'] = 'white'
rcParams['patch.edgecolor'] = 'white'

from matplotlib import font_manager as fm
proptease = fm.FontProperties()

### Function: flatlist_dropdup

In [14]:
def flatlist_dropdup(list_of_lists):
    return list(set([str(item) for sublist in list_of_lists for item in sublist]))

### Function: de_unicodeify

In [15]:
def de_unicodeify(data):
    if isinstance(data, basestring):
        return str(data)
    elif isinstance(data, collections.Mapping):
        return dict(map(de_unicodeify, data.iteritems()))
    elif isinstance(data, collections.Iterable):
        return type(data)(map(de_unicodeify, data))
    else:
        return data

***
## MAPPING TOOLS

### Function: kegg_mapper
#### KEGG ORGANISM & MAP TO DATABASE ID -> DICTIONARY OF MAPPED GENE IDS
This function simply creates a dictionary that maps KEGG IDs to either the **NCBI Entrez Gene ID** or the **UniProt ID**. It works using the KEGG REST API documented here: http://www.kegg.jp/kegg/docs/keggapi.html

**INPUTS**:
1. kegg_organism
    * Pick the one the SBML model is built off of from here: http://www.genome.jp/kegg/catalog/org_list.html
    * E. coli - 'eco'
    * Homo sapiens - 'hsa'
    * Thermotoga maritima (1999) - 'tma'
2. map_db
    * UniProt - 'uniprot'
    * NCBI Entrez - 'ncbi-geneid'

In [16]:
def kegg_mapper(kegg_organism, map_db='uniprot'):
    '''
    Generates a dictionary that maps KEGG gene IDs to NCBI Entrez gene IDs ('ncbi-geneid') or UniProt ('uniprot').
    Input:  kegg_organism - the KEGG organism name which you can determine from http://www.genome.jp/kegg/catalog/org_list.html
            map_db - ncbi-geneid OR uniprot (default): the database you want to map to
    Output: id_mapper - a dictionary of {KEGG_ID: mapped_ID}
    '''

    id_mapper = {}
    r = requests.post('http://rest.kegg.jp/conv/%s/%s' % (map_db,kegg_organism))

    for line in r.content.split('\n'):
        if not line: continue

        idents = line.split('\t')

        orig = idents[0].split(':')
        orig_database = orig[0]
        orig_id = orig[1]

        conv = idents[1].split(':')
        conv_database = conv[0]
        conv_id = conv[1]

        id_mapper[orig_id] = conv_id

    return id_mapper

### Function: bioservices_uniprot_mapper

In [17]:
def bioservices_uniprot_mapper(map_from, map_to, ident):
    return de_unicodeify(dict(bsup.mapping(fr=map_from, to=map_to, query=ident)))

### Function: uniprot_reviewed_checker

In [18]:
# status can be "unreviewed" or "reviewed"

def uniprot_reviewed_checker(uniprot_ids):
    # splitting query up into managable sizes (200 IDs each)
    Nmax = 200
    N, rest = divmod(len(uniprot_ids), Nmax)

    uni_rev_dict = {}

    if rest>0:
        N+=1
    for i in range(0,N):
        i1 = i*Nmax
        i2 = (i+1)*Nmax
        if i2>len(uniprot_ids):
            i2 = len(uniprot_ids)

        query = uniprot_ids[i1:i2]

        query_string = ''
        for x in query:
            query_string += 'id:' + x + '+OR+'
        query_string = query_string.strip('+OR+')
    
        uni_rev_raw = sio.StringIO(bsup.search(query_string, columns='id,reviewed', frmt='tab'))
        uni_rev_df = pd.read_table(uni_rev_raw, sep='\t', index_col=0)
        uni_rev_dict_adder = uni_rev_df.to_dict()['Status']
        uni_rev_dict.update(uni_rev_dict_adder)
        
        clear_output(wait=True)
        sys.stdout.flush()
    clear_output(wait=True)
    sys.stdout.flush()
    
    return de_unicodeify(uni_rev_dict)

### Function: pdb_current_checker

In [19]:
theoretical_pdbs_link = "ftp://ftp.wwpdb.org/pub/pdb/data/structures/models/index/titles.idx"

response = urllib2.urlopen(theoretical_pdbs_link)
theoretical_pdbs_raw = sio.StringIO(response.read())
theoretical_pdbs_df = pd.read_table(theoretical_pdbs_raw, sep='\t', header=None)

PDB_THEORETICAL = theoretical_pdbs_df[0].tolist()

In [20]:
PDB_OBSOLETE_MAPPING = {}

req = urllib2.Request('ftp://ftp.wwpdb.org/pub/pdb/data/status/obsolete.dat')
response = urllib2.urlopen(req)
for line in response:
    entry = line.split()
    if entry[0] == 'LIST':
        continue
    if len(entry) == 3:
        PDB_OBSOLETE_MAPPING[entry[2].upper()] = 'obsolete'
    if len(entry) >= 4:
        new = [y.upper() for y in entry[3:]]
        PDB_OBSOLETE_MAPPING[entry[2].upper()] = new

In [21]:
# status can be "theoretical", "current", or a PDB ID giving the non-obsolete entry

def pdb_current_checker(pdb_ids):
    if isinstance(pdb_ids, str):
        pdb_ids = [pdb_ids]
    
    pdb_status = {}
    
    theoretical = list(set(PDB_THEORETICAL).intersection(pdb_ids))
    obsolete = list(set(PDB_OBSOLETE_MAPPING.keys()).intersection(pdb_ids))
    current = list(set(pdb_ids).difference(theoretical,obsolete))
    
    for t in theoretical:
        pdb_status[t] = 'theoretical'
    for o in obsolete:
        pdb_status[o] = PDB_OBSOLETE_MAPPING[o]
    for c in current:
        pdb_status[c] = 'current'
        
    return pdb_status

### Internal use: SIFTS file

In [22]:
import os.path, time
now = time.time()
twodays_ago = now - 60*60*24*2

In [23]:
baseURL = "ftp://ftp.ebi.ac.uk/pub/databases/msd/sifts/flatfiles/csv/"
filename = "pdb_chain_uniprot.csv.gz"

final_filename = tempfile.gettempdir() + '/' + filename.split('.gz')[0]

if os.path.exists(final_filename):
    final_filename_creation_time = os.path.getmtime(final_filename)
    
    if final_filename_creation_time < twodays_ago:
        response = urllib2.urlopen(baseURL + filename)
        compressedFile = sio.StringIO(response.read())
        decompressedFile = gzip.GzipFile(fileobj=compressedFile)

        with open(final_filename, 'w') as outfile:
            outfile.write(decompressedFile.read())

        SIFTS = pd.read_csv(final_filename, skiprows=1, index_col=['PDB','CHAIN'])
        
    else:
        SIFTS = pd.read_csv(final_filename, skiprows=1, index_col=['PDB','CHAIN'])
else:
    response = urllib2.urlopen(baseURL + filename)
    compressedFile = sio.StringIO(response.read())
    decompressedFile = gzip.GzipFile(fileobj=compressedFile)

    with open(final_filename, 'w') as outfile:
        outfile.write(decompressedFile.read())

    SIFTS = pd.read_csv(final_filename, skiprows=1, index_col=['PDB','CHAIN'])

In [24]:
SIFTS['PDB_BEG_INT'] = SIFTS['PDB_BEG'].replace(to_replace=r'[^\d-]+', value='', regex=True)
SIFTS['SP_BEG_INT'] = SIFTS['SP_BEG'].replace(to_replace=r'[^\d-]+', value='', regex=True)

In [25]:
SIFTS['OFFSET'] = SIFTS['SP_BEG_INT'].astype(int) - SIFTS['PDB_BEG_INT'].astype(int)

### Function: sifts_pdb_chain_to_uniprot
#### PDB ID & CHAIN ID -> UNIPROT ID
This function uses the SIFTS mapping service to map PDB chains to their corresponding UniProt entries. PDB files have their own 'dbref' entry, however sometimes these map to obsolete UniProt entries.

**INPUTS**:
1. pdb
    * a string - PDB ID
2. chain
    * a string - chain ID

In [26]:
def sifts_pdb_chain_to_uniprot(pdb,chain):
    return SIFTS.ix[(pdb.lower(),chain.upper())]['SP_PRIMARY'].unique().tolist()

In [27]:
SIFTS.ix['4utq']

,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END,PDB_BEG_INT,SP_BEG_INT,OFFSET
CHAIN,,,,,,,,,,
A,P25714,1,548,1,548,1,548,1,1,0
Z,P68699,1,79,1,79,1,79,1,1,0


***
##UNIPROT RELATED

### Function: uniprot_valid_id

In [28]:
def uniprot_valid_id(test_id):
    # regex from: http://www.uniprot.org/help/accession_numbers
    valid_id = re.compile("[OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9]([A-Z][A-Z0-9]{2}[0-9]){1,2}")
    if valid_id.match(test_id): return True
    else: return False

### Function: uniprot_metadata
#### UNIPROT ID -> UNIPROT METADATA

This function retrieves data for a UniProt entry and returns a dictionary of that information.

**INPUT**:
1. uniprot_id

**OUTPUT**:
1. uniprot_metadata_dict
    * { 'UNIPROT': id,
        'DESCRIPTION': protein function,
        'KEGG': associated KEGG IDs,
        'SEQ': uniprot canonical sequence,
        'PDB': list of PDB IDs
      }

In [62]:
def parse_uniprot_txt_file(cache_txt):
    """
    Parses the text of metadata retrieved from uniprot.org.

    Only a few fields have been parsed, but this provides a
    template for the other fields.

    A single description is generated from joining alternative
    descriptions.

    Returns a dictionary with the main UNIPROT ACC as keys.
    """

    tag = None
    uniprot_id = None
    metadata_by_seqid = {}
    for l in cache_txt.splitlines():
        test_tag = l[:5].strip()
        if test_tag and test_tag != tag:
            tag = test_tag
        line = l[5:].strip()
        words = line.split()
        if tag == "ID":
            uniprot_id = words[0]
            is_reviewed = words[1].startswith('Reviewed')
            length = int(words[2])
            metadata_by_seqid[uniprot_id] = {
                'id': uniprot_id,
                'is_reviewed': is_reviewed,
                'length': length,
                'sequence': '',
                'accs': [],
            }
            entry = metadata_by_seqid[uniprot_id]
        if tag == "DT":
            #DT   01-OCT-1996, integrated into UniProtKB/Swiss-Prot.
            #DT   17-OCT-2006, sequence version 3.
            #DT   22-JUL-2015, entry version 166.
            comma_split = line.split(',')
            if 'sequence version' in comma_split[1]:
#                 print 'sequence_version', comma_split[0], dateparse(comma_split[0]).date()
                entry['sequence_version'] = str(dateparse(comma_split[0]).date())
            elif 'entry version' in comma_split[1]:
#                 print 'entry_version', comma_split[0], dateparse(comma_split[0]).date()
                entry['entry_version'] = str(dateparse(comma_split[0]).date())
        if tag == "SQ":
            if words[0] != "SEQUENCE":
                entry['sequence'] += ''.join(words)
        if tag == "AC":
            accs = [w.replace(";", "") for w in words]
            entry['accs'].extend(accs)
        if tag == "DR":
            if 'PDB' in words[0]:
                if 'pdb' not in entry:
                    entry['pdb'] = words[1][:-1]
                if 'pdbs' not in entry:
                    entry['pdbs'] = []
                entry['pdbs'].append(words[1][:-1])
            if 'RefSeq' in words[0]:
                if 'refseq' not in entry:
                    entry['refseq'] = []
                ids = [w[:-1] for w in words[1:]]
                entry['refseq'].extend(ids)
            if 'KEGG' in words[0]:
                if 'kegg' not in entry:
                    entry['kegg'] = []
                ids = [w[:-1] for w in words[1:]]
                ids = filter(lambda w: len(w) > 1, ids)
                entry['kegg'].extend(ids)
            if 'GO' in words[0]:
                if 'go' not in entry:
                    entry['go'] = []
                entry['go'].append(' '.join(words[1:]))
            if 'Pfam' in words[0]:
                if 'pfam' not in entry:
                    entry['pfam'] = []
                entry['pfam'].append(words[1][:-1])
        if tag == "GN":
            if 'gene' not in entry and len(words) > 0:
                pieces = words[0].split("=")
                if len(pieces) > 1 and 'name' in pieces[0].lower():
                    entry['gene'] = pieces[1].replace(';', '').replace(',', '')
        if tag == "OS":
            # OS   Homo sapiens (Human).
            if 'organism' not in entry:
                entry['organism'] = ""
            entry['organism'] += line
        if tag == "DE":
            if 'descriptions' not in entry:
                entry['descriptions'] = []
            entry['descriptions'].append(line)
        if tag == "CC":
            if 'comment' not in entry:
                entry['comment'] = ''
            entry['comment'] += line + '\n'

    for entry in metadata_by_seqid.values():
        descriptions = entry['descriptions']
        for i in reversed(range(len(descriptions))):
            description = descriptions[i]
            if 'Short' in description or 'Full' in description or 'EC=' in description:
                if 'Short' in description or 'Full' in description:
                    j = description.find('=')
                    descriptions[i] = description[j+1:].replace(';', '')
                    if 'description' not in entry:
                        entry['description'] = []
                    entry['description'].append(descriptions[i])
                if 'EC=' in description:
                    j = description.find('=')
                    descriptions[i] = description[j+1:].replace(';', '')
                    if '{' in descriptions[i]:
                        descriptions[i] = descriptions[i].split(' {')[0]
                    if 'ec' not in entry:
                        entry['ec'] = []
                    entry['ec'].append(descriptions[i])
            else:
                del descriptions[i]
    
    return metadata_by_seqid

In [61]:
def uniprot_metadata(uniprot_ids):
    '''
    Input: UniProt ID or IDs
    Output: dictionary of metadata associated with the UniProt IDs
    '''
    counter = 1
    
    if isinstance(uniprot_ids, str):
        uniprot_ids = [uniprot_ids]
        single = True
    else: single = False
    
    uniprot_metadata_raw = bsup.retrieve(uniprot_ids, frmt='txt')
    uniprot_metadata_final = {}
    
    for uniprot_id in uniprot_ids:
        
        clear_output(wait=True)
        print('***PROGRESS: %s - PARSED %d/%d UNIPROT IDS***\n' % (uniprot_id, counter, len(uniprot_ids)))
        counter += 1
        sys.stdout.flush()
        
        uniprot_metadata_dict = {}
        
        if single:
            metadata = parse_uniprot_txt_file(uniprot_metadata_raw)
        else:
            metadata = parse_uniprot_txt_file(uniprot_metadata_raw[uniprot_ids.index(uniprot_id)])
            
        metadata_key = metadata.keys()[0]
    
        uniprot_metadata_dict['u_uniprot_acc'] = uniprot_id
        uniprot_metadata_dict['u_seq'] = metadata[metadata_key]['sequence']
        uniprot_metadata_dict['u_seq_len'] = len(str(metadata[metadata_key]['sequence']))
        uniprot_metadata_dict['u_reviewed'] = metadata[metadata_key]['is_reviewed']
        uniprot_metadata_dict['u_seq_version'] = metadata[metadata_key]['sequence_version']
        uniprot_metadata_dict['u_entry_version'] = metadata[metadata_key]['entry_version']
        if 'gene' in metadata[metadata_key]:
            uniprot_metadata_dict['u_gene_name'] = metadata[metadata_key]['gene']
        if 'description' in metadata[metadata_key]:
            uniprot_metadata_dict['u_description'] = metadata[metadata_key]['description']
        if 'refseq' in metadata[metadata_key]:
            uniprot_metadata_dict['u_refseq'] = metadata[metadata_key]['refseq']
        if 'kegg' in metadata[metadata_key]:
            uniprot_metadata_dict['u_kegg_id'] = metadata[metadata_key]['kegg']
        if 'go' in metadata[metadata_key]:
            uniprot_metadata_dict['u_go'] = metadata[metadata_key]['go']
        if 'ec' in metadata[metadata_key]:
            uniprot_metadata_dict['u_ec_number'] = metadata[metadata_key]['ec']
        if 'pfam' in metadata[metadata_key]:
            uniprot_metadata_dict['u_pfam'] = metadata[metadata_key]['pfam']
        
        uniprot_metadata_final[uniprot_id] = uniprot_metadata_dict
    return de_unicodeify(uniprot_metadata_final)

### Function: uniprot_reviewed_mapper_plus_metadata
#### ANY ID & INPUT TYPE -> UNIPROT METADATA

This function maps an ID to UniProt, retrieves the metadata for the mapped ID, and returns a dictionary of that information.

**INPUT**:
1. gene_id
2. id_type

**OUTPUT**:
1. uniprot_metadata_dict
    * { 'UNIPROT': id,
        'DESCRIPTION': protein function,
        'KEGG': associated KEGG IDs,
        'SEQ': uniprot canonical sequence,
        'PDB': list of PDB IDs
      }

In [25]:
def uniprot_reviewed_mapper_plus_metadata(gene_id, id_type):
    '''
    Input: Any ID type specified from: http://www.uniprot.org/faq/28#id_mapping_examples
    reviewed=True for only reviewed UniProt entries
    '''

    uniprot_metadata_dict = {}

    reviewed_uniprots = uniprot_mapper(gene_id, id_type, reviewed=True)

    if len(reviewed_uniprots) > 1:
        warnings.warn('%s: MORE THAN 1 REVIEWED UNIPROT ENTRY' % gene_id)

    uniprot_id = reviewed_uniprots[0]

    return uniprot_metadata(uniprot_id)

### Function: uniprot_ec
#### UNIPROT ID -> EC NUMBER

This function returns the EC number associated with a UniProt ID

**INPUT**:
1. uniprot_id

**OUTPUT**:
1. EC number (as a string)

In [26]:
def uniprot_ec(uniprot_id):
    r = requests.post('http://www.uniprot.org/uniprot/?query=%s&columns=ec&format=tab' % uniprot_id)

    ec = r.content.splitlines()[1]

    if len(ec) == 0:
        ec = None

    return ec

### Function: uniprot_sites

In [27]:
def uniprot_sites(uniprot_id):
    r = requests.post('http://www.uniprot.org/uniprot/%s.gff' % uniprot_id)
    gff = sio.StringIO(r.content)
    
    gff_df = pd.read_table(gff, sep='\t',skiprows=2,header=None)
    gff_df.drop([0,1,5,6,7,9], axis=1, inplace=True)
    gff_df.columns = ['type','seq_start','seq_end','notes']
    
    return gff_df

***
##PDB RELATED

In [57]:
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB import PDBIO
keep_alt=0
# create class to select non disordered atoms and those with the correct alternate location ID
class NotDisordered(PDB.Select):
  def __init__(self,alt):
    self.alt=alt
  def accept_atom(self,atom):
    if not atom.is_disordered():
      return True
    elif atom.get_altloc() == self.alt:
      if not keep_alt:
        atom.set_altloc(' ')
      return True
    else:
      return False

### Function: clean_pdb

Adds chains and occupancies to incomplete PDB files. Used for automatically generated homology models that don't have these, since STRIDE, DSSP, biopython, and others will complain about these missing components.

In [ ]:
def clean_pdb(input_pdb):
    
    # default is to keep atoms with alternate location ID 'A'
    alt = 'A'
    # default is to not keep "alt" identifier
    keep_alt = 0
    
    parser = PDB.PDBParser()
    
    filename = input_pdb.split('.pdb')[0]
    occ_beta_added = filename + '_fix.pdb'

    struct = parser.get_structure(filename, input_pdb)

    # adding chain X if the chain column is empty (http://comments.gmane.org/gmane.comp.python.bio.devel/10639)
    for x in struct.get_chains():
        if not x.id.strip(): # chain could be an empty string ' ' so strip it!
            x.id = 'X'

    # adding occupancies if there are none (http://comments.gmane.org/gmane.comp.python.bio.general/6289)
    for atom in struct.get_atoms():
        atom.set_occupancy(1)

    # create PDBIO class for writing
    io=PDB.PDBIO()
    io.set_structure(struct)
    # save it
    io.save(occ_beta_added, select=NotDisordered(alt))
    
    return occ_beta_added

### Function: get_pdb_seq
#### 2015-02-10 in progress

trying to test biopython 

resources:

* http://mmcif.wwpdb.org/pdbx-mmcif-home-page.html
* http://biopython.org/wiki/The_Biopython_Structural_Bioinformatics_FAQ#I.27d_like_to_have_some_more_low_level_access_to_an_mmCIF_file...

In [28]:
def get_pdb_seq(pdb_id, pdb_file):
    '''
    this uses biopython to get the sequence of the residues that are resolved in the structure. 
    IT ADDS AN X IN POSITIONS THAT HAVE NO RESOLVED RESIDUE!
    IT EVEN WORKS WITH CIF FILES!
    '''
    
    if pdb_file.endswith('.pdb'):
        parser = PDBParser()
        structure = parser.get_structure(pdb_id, pdb_file)
    elif pdb_file.endswith('.cif'):
        parser = MMCIFParser()
        try:
            structure = parser.get_structure(pdb_id, pdb_file)
        except:
            return None
        
    structure_seqs = []
    for chain in structure[0]:
        chain_seq = SeqRecord(Seq('', IUPAC.protein), name=pdb_id, id=pdb_id+'.'+chain.get_id(), description='PDB structure residues')
        first = True
        tracker = 0
        for res in chain.get_residues():
            if is_aa(res, standard=True):
                full_id = res.get_full_id()
                end_tracker = full_id[3][1]
                i_code = full_id[3][2]
                aa = three_to_one(res.get_resname())
                
                # a bit odd -- shouldnt i check if end_tracker = tracker first?
                # then if the i_code is ' ', do the same thing
                # and if it is not, it is because the last residue had an i_code so we just need to add this residue
                # right??
                # as it currently is it adds -1 X's (none) if there is a previous insertion code, which works
                if end_tracker != (tracker + 1) and first == False:
                    if i_code != ' ':
                        chain_seq += aa
                        tracker = end_tracker + 1
                        continue

                    else:
                        chain_seq += 'X'*(end_tracker - tracker - 1)
                        
                chain_seq += aa
                first = False
                tracker = end_tracker

        structure_seqs.append(chain_seq)

    return structure_seqs


## ALTERNATE BIOPYTHON STRUCTURE SEQUENCE
# using the polypeptide builder to get the residues
# works BUT if you cannot build a polypeptide (ie there is a "floating" residue somewhere), it ignores that residue
# example is with mmCIF but can use a pdb too

# from Bio.PDB import *
# parser = MMCIFParser()
# structure = parser.get_structure('1seh', '1SEH.cif')

# ppb=PPBuilder()
# for pp in ppb.build_peptides(structure): 
#     print pp
#     seq = pp.get_sequence()
#     print seq

# for model in structure:
#     for chain in model:
#         chain_seq = ''

#         first = True
#         end_tracker = 0
        
#         for pp in ppb.build_peptides(chain):
#             start = pp[0].get_id()[1]
#             end = pp[-1].get_id()[1]
#             print pp, start, end
#             seq = pp.get_sequence()
#             print seq
            
#             if start != end_tracker - 1 and first == False:
#                 print start - end_tracker - 1
#                 chain_seq += 'X'*(start - end_tracker - 1)
            
#             chain_seq += seq
                
#             first=False
#             end_tracker = end
            
#         print chain_seq

In [29]:
def get_pdb_seq_prody(pdbid):
    # TODO: ABOUT TO DEPRECATE THIS ONE
    '''
    This function downloads a PDB ID into the working directory and also
    returns the sequence that is represented in the structure (what residues are visible).
    The sequence is a SeqRecord Biopython object.

    Input: pdbid - a 4 letter string
    Output: structure_seqs - a Seq object that represents what residues can be seen in the structure
            resolution - the crystal structure resolution
    '''
    
    try:
        pdb = pr.parsePDB(pdbid, header=False, biomol=False, secondary=False)
        
        # if multiple biomols choose the first one (this could be improved)
        if type(pdb) == list:
            pdb = pdb[0]
            
    # if PDB isn't able to be downloaded, try downloading the mmCIF
    # TODO: how to parse structure information from mmCIF? returning nothing for now, just downloading it
    except IOError:
        try:
            pr.fetchPDBviaFTP(pdbid,format='cif')
            return None
        except IOError:
            return None

    structure_seqs = []
    for chain in pdb.iterChains():
        ch = chain.getChid()
        seq = SeqRecord(Seq(chain.getSequence(), IUPAC.protein), id=pdbid+'.'+ch, description='PDB structure residues')
        structure_seqs.append(seq)

    return structure_seqs

In [30]:
def get_pdb_res_starts(pdb_id, pdb_file):
    '''
    this uses biopython to get the first residue number in a pdb file. 
    returns a list of tuples of pdb, chain, resnum. 

    '''
    if pdb_file.endswith('.pdb'):
        parser = PDBParser()
        structure = parser.get_structure(pdb_id, pdb_file)
    if pdb_file.endswith('.cif'):
        parser = MMCIFParser()
        structure = parser.get_structure(pdb_id, pdb_file)
        
    start_residues = []
    for chain in structure[0]:
        residues = chain.get_residues()
        start_residues.append((pdb_id, chain.get_id(), residues.next().get_id()[1]))
    
    return start_residues

In [31]:
def pdb_metadata_and_download(pdbids):
    final_dict = {}
    counter = 1
    
    if isinstance(pdbids, str):
        pdbids = [pdbids]
    
    for pdbid in pdbids:
        
        clear_output(wait=True)
        print('***PROGRESS: PARSED & DOWNLOADED %d/%d PDB IDS***\n' % (counter, len(pdbids)))
        counter += 1
        sys.stdout.flush()
        
        try:
            pr.fetchPDB(pdbid, compressed=False)
            header = pr.parsePDBHeader(pdbid)

        # if PDB isn't able to be downloaded
        except IOError:
            try:
                header = pr.parsePDBHeader(pr.fetchPDBviaFTP(pdbid,format='cif', compressed=False))
            except IOError:
                continue
        
        appender = defaultdict(list)
        for prop,val in header.iteritems():
            if isinstance(val, pr.proteins.header.Chemical):
                appender['p_chemicals'].append(val.resname)
            elif isinstance(val, pr.proteins.header.Polymer):
                appender['p_chains'].append(val.chid)
                if val.ec:
                    appender['p_ec_numbers'].append((val.chid,val.ec))
            elif prop == 'reference':
                if 'doi' in val:
                    appender['p_doi'] = val['doi']
                if 'pmid' in val:
                    appender['p_pmid'] = val['pmid']
            elif prop in ['resolution','space_group','experiment','deposition_date']:
                appender['p_' + prop] = val
        
        tmp = {}
        for chain in appender['p_chains']:
            try:
                uniprot_mapping = sifts_pdb_chain_to_uniprot(pdbid, chain)
            except KeyError:
                uniprot_mapping = ['PDB-'+chain]
            tmp[chain] = uniprot_mapping
        appender['p_chain_uniprot_map'] = tmp

        final_dict[pdbid] = dict(appender)
            
    return de_unicodeify(final_dict)

In [32]:
AAdict={'MSE': 'M', 'ALA': 'A', 'LEU': 'L', 'MET': 'M', 'ARG': 'R', 'LYS': 'K', 'GLN': 'Q', 'GLU': 'E', 'ILE': 'I', 'SER': 'S', 'TRP': 'W', 'TYR': 'Y', 'PHE': 'F', 'VAL': 'V', 'THR': 'T', 'HIS': 'H', 'CYS': 'C', 'ASN': 'N', 'ASP': 'D', 'GLY': 'G', 'PRO': 'P'}

AAdict2 = {'C':'CYS',
 'I':'ILE',
 'G':'GLY', 
 'S':'SER', 
 'Q':'GLN', 
 'K':'LYS',
 'N':'ASN',
 'P':'PRO', 
 'D':'ASP', 
 'T':'THR', 
 'F':'PHE', 
 'A':'ALA', 
 'M':'MET', 
 'H':'HIS', 
 'L':'LEU', 
 'R':'ARG', 
 'W':'TRP', 
 'V':'VAL', 
 'E':'GLU', 
 'Y':'TYR'}

keep_atom_list = ['N', 'C', 'O', 'CA']

In [56]:
def setup_pdb_for_amber(df, name, file_name):
    
    ''' takes in a PDB or homology file and (1) performs site-directed mutagenesis given a user-defined sequence change,
    (2) removes all hetero atoms (e.g. water and metals/cofactors) and (3) removes disordered atoms
    '''
    
    parser = PDBParser()
    structure = parser.get_structure(name, file_name)
    model = structure[0]
    chains = [i.id for i in model.child_list]

    for chain_id in chains:
        chain = model[chain_id]
        residue_list = chain.child_list

        for residue in list(residue_list):
            # mutate residues according to sequence alignment data
#             print residue.id[1]
            if len(df)>0:
                if residue.id[1] in df[df.id_b.str.endswith(chain_id)].id_b_start.tolist():
                
    #                 print name, residue.id[1], chain_id
    #                 print df
                    res_w = df[df.id_b.str.endswith(chain_id)][df.id_b_start == residue.id[1]].id_b_aa.values[0]
                    res_mut = df[df.id_b.str.endswith(chain_id)][df.id_b_start == residue.id[1]].id_a_aa.values[0]

                    if res_mut not in AAdict2.keys():
                        warnings.warn('***UNKNOWN AMINO ACID IN UNIPROT SEQUENCE. SKIPPING***') ### TO CHANGE!!!!!!!!!!!!!!!
                        continue

                    print df[df.id_b.str.endswith(chain_id)][df.id_b_start == residue.id[1]].values 
                    print '\n'
                    print residue.id[1], residue.resname.upper()," | ", res_w, "mutate to:", res_mut

                    # Remove all atoms except protein backbone atoms:
                    for atom in residue.child_dict.keys():
                        if atom in keep_atom_list:
                            pass
                        else:
                            residue.detach_child(atom)

                    residue.resname = AAdict2[res_mut]
                    # if residue is non-standard, make it a non-heteroatom
                    if residue.id[0] != ' ':
                        resnum = residue.id[1]
                        residue.id = (' ',resnum, ' ')
                    # check that its the residue you expect and rename the residue:  
    #                 if AAdict[residue.resname] == res_w:#AAdict[res_w]:
    #                     print 'matches \n'
    #                     residue.resname = AAdict2[res_mut]
    #                     print residue.resname
    #                 else:
    #                     raise ValueError("Unrecognised residue %r" % residue.resname)
            
        # Remove all hydrogens and heteroatom residues (e.g. WAT or metal) from structure:  
        for residue in list(chain):
            id = residue.id
            if id[0] != ' ':
                chain.detach_child(id)
            if len(chain) == 0:
                model.detach_child(chain.id)
            for atom in residue.get_list():
            #print residue.resname, residue.id[1], atom.element
                if atom.element == 'H':
                    residue.detach_child(atom.id)
                

    return structure

def write_to_pdb(struct, new_file_name):
    w = PDBIO()
    w.set_structure(struct)
    w.save(new_file_name)

In [2]:
def final_mutation_function(corrected_df, pdb_id, pdb_file):
    # default is to keep atoms with alternate location ID 'A'
    alt = 'A'
    # default is to not keep "alt" identifier
    keep_alt = 0
    
    output = pdb_id + '_modified.pdb'
#     output = pdb_id + '.pdb'
    
    #read in structure to parser
    parser = PDB.PDBParser()
    struct = parser.get_structure(pdb_id, pdb_file)
    
    # create PDBIO class for writing
    io = PDB.PDBIO()
    io.set_structure(struct)
    # save it
    io.save('/tmp/%s_no_disorder.pdb' % pdb_id, select=NotDisordered(alt))
    
    # choose only parts of the protein to mutate
    df = corrected_df[corrected_df.type == 'mutation']
    # load structure (no disorder) into biopython & perform modifications
    structure_new = setup_pdb_for_amber(df, pdb_id, '/tmp/%s_no_disorder.pdb' % pdb_id)
    #  write out new structure
    write_to_pdb(structure_new, output)
    
    return output

In [6]:
def setup_pdb_for_amber_monomer(df, name, file_name):
    
    ''' takes in a PDB or homology file and (1) performs site-directed mutagenesis given a user-defined sequence change,
    (2) removes all hetero atoms (e.g. water and metals/cofactors) and (3) removes disordered atoms
    MOD: keep only 1 chain (df is already the "corrected" df with only the gene of interest anyway)
    '''
    
    parser = PDBParser()
    structure = parser.get_structure(name, file_name)
    model = structure[0]
#     chains = [i.id for i in model.child_list]

    # get mutations, if any
    mutation_df = df[df.type == 'mutation']
    
    # choosing a representative chain
    sorted_by_match_and_chain = df[df['type']=='match'].sort(['count','chain'], ascending=[False, True]).set_index(['chain','count']).index.tolist()
    chain_id = str(sorted_by_match_and_chain[0][0])
    
    # operate only on this chain
    chain = model[chain_id]
    residue_list = chain.child_list

    for residue in list(residue_list):
        # mutate residues according to sequence alignment data
#             print residue.id[1]
        if len(mutation_df)>0:
            if residue.id[1] in df[df.id_b.str.endswith(chain_id)].id_b_start.tolist():

#                 print name, residue.id[1], chain_id
#                 print df
                res_w = df[df.id_b.str.endswith(chain_id)][df.id_b_start == residue.id[1]].id_b_aa.values[0]
                res_mut = df[df.id_b.str.endswith(chain_id)][df.id_b_start == residue.id[1]].id_a_aa.values[0]

                if res_mut not in AAdict2.keys():
                    warnings.warn('***UNKNOWN AMINO ACID IN UNIPROT SEQUENCE. SKIPPING***') ### TO CHANGE!!!!!!!!!!!!!!!
                    continue

                print df[df.id_b.str.endswith(chain_id)][df.id_b_start == residue.id[1]].values 
                print '\n'
                print residue.id[1], residue.resname.upper()," | ", res_w, "mutate to:", res_mut

                # Remove all atoms except protein backbone atoms:
                for atom in residue.child_dict.keys():
                    if atom in keep_atom_list:
                        pass
                    else:
                        residue.detach_child(atom)

                residue.resname = AAdict2[res_mut]
                # if residue is non-standard, make it a non-heteroatom
                if residue.id[0] != ' ':
                    resnum = residue.id[1]
                    residue.id = (' ',resnum, ' ')
                # check that its the residue you expect and rename the residue:  
#                 if AAdict[residue.resname] == res_w:#AAdict[res_w]:
#                     print 'matches \n'
#                     residue.resname = AAdict2[res_mut]
#                     print residue.resname
#                 else:
#                     raise ValueError("Unrecognised residue %r" % residue.resname)

    # Remove all hydrogens and heteroatom residues (e.g. WAT or metal) from structure:  
    for residue in list(chain):
        id = residue.id
        if id[0] != ' ':
            chain.detach_child(id)
        if len(chain) == 0:
            model.detach_child(chain.id)
        for atom in residue.get_list():
        #print residue.resname, residue.id[1], atom.element
            if atom.element == 'H':
                residue.detach_child(atom.id)


    return chain

def write_to_pdb(struct, new_file_name):
    w = PDBIO()
    w.set_structure(struct)
    w.save(new_file_name)

In [5]:
def monomer_or_mutation_function(corrected_df, pdb_id, pdb_file):
    # default is to keep atoms with alternate location ID 'A'
    alt = 'A'
    # default is to not keep "alt" identifier
    keep_alt = 0
    
    output = pdb_id + '_modified.pdb'
#     output = pdb_id + '.pdb'
    
    #read in structure to parser
    parser = PDB.PDBParser()
    struct = parser.get_structure(pdb_id, pdb_file)
    
    # create PDBIO class for writing
    io = PDB.PDBIO()
    io.set_structure(struct)
    # save it
    io.save('/tmp/%s_no_disorder.pdb' % pdb_id, select=NotDisordered(alt))
    
    # load structure (no disorder) into biopython & perform modifications
    structure_new = setup_pdb_for_amber_monomer(corrected_df, pdb_id, '/tmp/%s_no_disorder.pdb' % pdb_id)
    #  write out new structure
    write_to_pdb(structure_new, output)
    
    return output

### Function: pdb_chain_stoichiometry_biomolone
#### PDB ID -> DOWNLOAD/PARSE PDB & CHAIN STOICHIOMETRY

In [35]:
def pdb_chain_stoichiometry_biomolone(pdbid):
    '''
    Takes in a PDB ID and returns the stoichiometry of the chains in biological assembly 1 as a dictionary.
    Steps taken are:
    1) download PDB and parse header, make biomolecule if provided
    2) count how many times each chain appears in biomolecule #1
    3) convert chain id to uniprot id
    4) return final dictionary
    Input: pdbid - a 4 character PDB ID
    Output: a dictionary of {(ChainID,UniProtID): # occurences}
    '''
    
    try:
        # change to biomol=True if you want biological assemblies
        pdb, header = pr.parsePDB(pdbid, header=True, biomol=True, secondary=False)

        # if multiple biomols choose the first one, also if NMR choose the first coord set
        if type(pdb) == list:
            pdb = pdb[0]

    except IOError:
        return None, None
    # if there are no biological assemblies
    except ValueError:
        pdb, header = pr.parsePDB(pdbid, header=True, biomol=False, secondary=False)
    
    # count the occurences of each chain
    chain_stoich = defaultdict(int)
    hier = pdb.getHierView()
    for chain in hier:
        chain_stoich[chain.getChid()] += 1
    
    # DBREF entry in PDB file sometimes contains obsolete UniProt entries
    # chain_to_uniprot = {}
    # for chain in header['polymers']:
        # for dbref in chain.dbrefs:
            # if dbref.database.lower() == 'uniprot':
                # chain_to_uniprot[chain.chid] = dbref.accession
    
    # convert chain IDs to uniprot IDs
    chain_to_uniprot = {}
    for chain in header['polymers']:
        try:
            chain_to_uniprot[chain.chid] = sifts_pdb_chain_to_uniprot(pdbid.lower(), chain.chid)
        except KeyError:
            chain_to_uniprot[chain.chid] = ['PDB-'+chain.chid]
                
    # keep both chain ID and uniprot ID (this is the final dictionary)
    combined = {}
    for k,v in chain_to_uniprot.iteritems():
        for uni in v:
            combined[(k,uni)] = chain_stoich[k]
    
    return combined

### Function: pisa_complex_information
#### PDB ID -> PREDICTED PISA COMPLEXES

In [36]:
def pisa_complex_information(pdb_id):
    
    pdb_id = pdb_id.lower()
    pisa = {}
    
    # request the xml file for a PDB
    r = requests.post('http://www.ebi.ac.uk/pdbe/pisa/cgi-bin/multimers.pisa?%s' % pdb_id)
    soup = BeautifulSoup(r.content)
    
    # if PISA can't calculate an assembly...
    if 'not found' in str(soup.pdb_entry.status.string) or 'No symmetry' in str(soup.pdb_entry.status.string) or 'Overlapping structures' in str(soup.pdb_entry.status.string):
        pisa[pdb_id] = 'ERROR'
        return pisa
    
    # if it is a monomer...
    num_complexes = int(soup.total_asm.string)
    if num_complexes == 0:
        pisa[pdb_id] = 'MONOMER'
        return pisa
        
    # otherwise, get the complex information!
    elif num_complexes > 0:
        
        # all "assembly sets" (see PISA sets for more info)
        sets = soup.findAll('asm_set')
        
        for s in sets:
            
            set_id = int(s.ser_no.string)
            
            # all assemblies
            complexes = s.findAll('assembly')
            
            for cplx in complexes:
                
                ############################################################################################
                # this part tells you the actual composition of the predicted complex (chains and ligands) #
                parts = cplx.findAll('molecule')

                chains = defaultdict(int)

                for part in parts:
                    part_id = part.chain_id.string
                    if part_id.startswith('['):
                        part_id = 'LIG_' + part_id.split(']')[0].strip('[')
                    chains[str(part_id)] += 1

                ligands = {}

                for key in chains.keys():
                    if key.startswith('LIG_'):
                        ligands[str(key.split('_')[1])] = chains.pop(key)
                        
                chains_final = {}
                for k,v in chains.iteritems():
                    try:
                        chain_to_uniprot = sifts_pdb_chain_to_uniprot(pdb_id.lower(), k)
                    except KeyError:
                        chain_to_uniprot = ['PDB-' + k]
                    for m in chain_to_uniprot:
                        chains_final[(str(k), str(m))] = v
                ############################################################################################
                
                # this part give you something to add to a dataframe
                adder = {}

                cplx_id = int(cplx.id.string)
                cplx_composition = str(cplx.composition.string)

                d_g_diss = float(cplx.diss_energy.string)
                d_g_int = float(cplx.int_energy.string)

                pdb_biomol = int(cplx.r350.string)
                
                if d_g_diss >= 0:
                    stable = True
                else:
                    stable = False

                adder['cplx_composition'] = cplx_composition.strip()
                adder['cplx_chains'] = chains_final
                adder['cplx_ligands'] = ligands
                adder['stable'] = stable
                adder['d_g_diss'] = d_g_diss
                adder['d_g_int'] = d_g_int
                adder['pdb_biomol'] = pdb_biomol
                
                pisa[(set_id,cplx_id)] = adder
    
        return pisa

***
## SEQUENCE TOOLS

### Function: write_fasta_file
#### This writes a fasta file for a SeqRecord object. It also checks if the file exists already and returns the filename.

**INPUTS**:
1. sequence
    * A Biopython SeqRecord object (that could contain multiple sequences in one)
2. identification
    * A string representing the ID (usually UniProt or PDB)
    
**OUTPUTS**
1. outfile
    * The filename of the fasta (.faa) file generated

In [37]:
def write_fasta_file(sequence, identification):
    '''
    This writes a fasta file for a SeqRecord object. It also checks if the file exists already and returns the filename.
    
    Input: sequence - Biopython SeqRecord object, identification - ID of the sequence.
    Output: Filename of fasta file
    '''
    
    from Bio import SeqIO
    import os.path
    
    outfile = "%s.faa" % identification
    if os.path.isfile(outfile):
        print 'FASTA file already exists %s' % outfile
        return outfile
    else:
        SeqIO.write(sequence, outfile, "fasta")
        return outfile

### Function: run_alignment
#### This runs the EMBOSS needle application (Needleman–Wunsch algorithm) to align a uniprot sequence to a PDB fasta sequence. It also checks if the file exists already and returns the filename.

**INPUTS**:
1. uniprot_faa
    * A Biopython SeqRecord object (usually just one sequence)
2. pdb_faa
    * A Biopython SeqRecord object (can have multiple because of multiple chains in a PDB structure)
    
**OUTPUTS**
1. alignment_file
    * The filename of the alignment file generated

In [60]:
def run_alignment(uni_id, uniprot_faa, pdb_id, pdb_faa):
    '''
    Runs the needle alignment program and writes the result to a file. Returns the filename.
    
    Input:  uniprot_faa - uniprot fasta file name
            pdb_faa - pdb fasta file name
    Output: alignment_file - file name of alignment
    '''
    
    from Bio.Emboss.Applications import NeedleCommandline
    import os.path

    alignment_file = "%s_%s_align.txt" % (uni_id, pdb_id)
    
    if os.path.isfile(alignment_file):
        print 'Alignment %s file already exists' % alignment_file
        return alignment_file

    else:
        # print '**RUNNING ALIGNMENT FOR %s AND %s**' % (uni_id, pdb_id)
        needle_cline = NeedleCommandline(asequence=uniprot_faa, bsequence=pdb_faa, gapopen=10, gapextend=0.5, outfile=alignment_file)
        stdout, stderr = needle_cline()
        return alignment_file

In [101]:
# SEE: https://www.biostars.org/p/91124/

def run_alignment2(a_id, a_seq, b_id, b_seq):
    '''
    Runs the needle alignment program and returns a raw text dump of the alignment
    
    Input:  a_id - sequence ID #1 (string)
            a_seq - sequence #1 (string)
            b_id - sequence ID #2 (string)
            b_seq - sequence #2 (string)
    Output: alignment_file - file name of alignment
    
    DEPENDENCIES:
    get_alignment_allpos_df
    '''
    
    from Bio.Emboss.Applications import NeedleCommandline
    from Bio import AlignIO
    import os.path

    alignment_file = "/tmp/%s_%s_align.txt" % (a_id, b_id)

    needle_cline = NeedleCommandline(asequence="asis::"+a_seq, bsequence="asis::"+b_seq, gapopen=10, gapextend=0.5, outfile=alignment_file)
    stdout, stderr = needle_cline()
    
    return get_alignment_allpos_df(alignment_file, a_id, b_id)

### Function: run_alignment_needleall
#### TODO: runs needleall!

In [61]:
def run_alignment_needleall(a_id, a_faa, b_id, b_faa):
    
    from Bio.Emboss.Applications import NeedleallCommandline
    import os.path

    alignment_file = "%s-%s_align.txt" % (a_id, b_id)
    
    if os.path.isfile(alignment_file):
        print 'Alignment %s file already exists' % alignment_file
        return alignment_file

    else:
        print '**RUNNING ALIGNMENT FOR %s AND %s**' % (a_id, b_id)
        needle_cline = NeedleallCommandline(asequence=a_faa, bsequence=b_faa, gapopen=10, gapextend=0.5, outfile=alignment_file)
        stdout, stderr = needle_cline()
        return alignment_file

### Function: needle_reader
#### Reads in a needle alignment file and spits out statistics of the alignment.

**INPUTS**:
1. fl
    * Filename of the alignment file

**OUTPUTS**
1. alignment_properties
    * Dictionary of dictionaries representing statistics of the alignment { 'PDB_A': {'gaps': ##, 'identity': ##, 'similarity': ##, 'score': ####}, 'PDB_B': ...}

In [39]:
def needle_reader(fl):
    '''
    Reads in a needle alignment file and spits out statistics of the alignment.
    
    Input: fl - alignment file name
    Output: alignment_properties - a dictionary telling you the number of gaps, identity, etc.
    '''
    
    alignments = list(AlignIO.parse(fl, "emboss"))
    
    f=open(fl)
    alignment_properties = defaultdict(dict)
    line = f.readline()

    for i in range(len(alignments)):
        #print alignments[i]
        while line.rstrip() != "#=======================================":
            line = f.readline()
            if not line:
                raise StopIteration

        while line[0] == "#":
            #Read in the rest of this alignment header,
            #try and discover the number of records expected
            #and their length
            parts = line[1:].split(":", 1)
            key = parts[0].lower().strip()
            if key == '2':
                pdb_id = parts[1].strip()
            if key == 'identity':
                ident_parse = parts[1].strip().replace('(','').replace(')','').replace('%','').split()
                ident_num = int(ident_parse[0].split('/')[0])
                ident_percent = float(ident_parse[1])
                alignment_properties[pdb_id]['identity'] = ident_num
                alignment_properties[pdb_id]['identity_percent'] = ident_percent
            if key == 'similarity':
                sim_parse = parts[1].strip().replace('(','').replace(')','').replace('%','').split()
                sim_num = int(sim_parse[0].split('/')[0])
                sim_percent = float(sim_parse[1])
                alignment_properties[pdb_id]['similarity'] = sim_num
                alignment_properties[pdb_id]['similarity_percent'] = sim_percent
            if key == 'gaps':
                gap_parse = parts[1].strip().replace('(','').replace(')','').replace('%','').split()
                gap_num = int(gap_parse[0].split('/')[0])
                gap_percent = float(gap_parse[1])
                alignment_properties[pdb_id]['gaps'] = gap_num
                alignment_properties[pdb_id]['gaps_percent'] = gap_percent
            if key == 'score':
                score = float(parts[1].strip())
                alignment_properties[pdb_id]['score'] = score
            
            #And read in another line...
            line = f.readline()

    return alignment_properties

### Function: get_alignment_df
#### Reads in a needle alignment file and creates a pandas dataframe telling you what the mutations are.

**INPUTS**:
1. TODO


**OUTPUTS**
1. TODO

In [67]:
def get_alignment_df(alignment_file):
    alignments = list(AlignIO.parse(alignment_file, "emboss"))
    alignment_df = pd.DataFrame(columns = ['id_a', 'id_b', 'type', 'id_a_start', 'id_a_stop', 'count', 'id_a_aa', 'id_b_aa'])

    for alignment in alignments:
    #         if not switch:
        a_seq_id = list(alignment)[0].id
        a_seq = str(list(alignment)[0].seq)
        b_seq_id = list(alignment)[1].id
        b_seq = str(list(alignment)[1].seq)
    #         else:
    #             a_seq_id = list(alignment)[1].id
    #             a_seq = str(list(alignment)[1].seq)
    #             b_seq_id = list(alignment)[0].id
    #             b_seq = str(list(alignment)[0].seq)

        idx_start = 1

        uniprot_shift = 0
        insertion_counter = 0
        for i, (a,b) in enumerate(zip(a_seq,b_seq)):
            if uniprot_shift != 0:
                new_i = i-uniprot_shift
            else:
                new_i = i

            if a == b and a != '-' and b != '-':
                aa_flag = 'match'
            if a != b and a == '-' and b != '-':
                aa_flag = 'insertion'
            if a != b and a != '-' and b == '-':
                aa_flag = 'deletion'
            if a != b and a != '-' and b == 'X':
                aa_flag = 'unresolved'
            elif a != b and a != '-' and b != '-':
                aa_flag = 'mutation'
            if i == 0:
                aa_flag_tracker = aa_flag

            if aa_flag == 'insertion':
                uniprot_shift += 1
                insertion_counter += 1


            if aa_flag != aa_flag_tracker:
                idx_end = new_i
                if aa_flag_tracker == 'match' or aa_flag_tracker == 'deletion' or aa_flag_tracker == 'unresolved':
                    appender = {}
                    appender['id_a'] = a_seq_id
                    appender['id_b'] = b_seq_id
                    appender['type'] = aa_flag_tracker
                    appender['id_a_start'] = idx_start
                    appender['id_a_stop'] = idx_end
                    appender['count'] = idx_end - idx_start + 1
#                     print '%s from %d to %d with %d members' % (appender['type'], appender['start'], appender['stop'], appender['count'])
                    alignment_df = alignment_df.append(appender, ignore_index = True)

                if aa_flag_tracker == 'insertion':
                    appender = {}
                    appender['id_a'] = a_seq_id
                    appender['id_b'] = b_seq_id
                    appender['type'] = aa_flag_tracker
                    appender['id_a_start'] = new_i
                    appender['id_a_stop'] = new_i
                    appender['count'] = insertion_counter
#                     print '%s of length %d' % (aa_flag_tracker, insertion_counter)
                    alignment_df = alignment_df.append(appender, ignore_index = True)
                    insertion_counter = 0
                idx_start = new_i + 1

            # HEY. THIS NEEDS TO BE OUTSIDE THE FOR LOOP
            if aa_flag == 'mutation':
                appender = {}
                appender['id_a'] = a_seq_id
                appender['id_b'] = b_seq_id
                appender['type'] = aa_flag
                appender['id_a_start'] = new_i + 1
                appender['id_a_stop'] = new_i + 1
                appender['count'] = 1
                appender['id_a_aa'] = a
                appender['id_b_aa'] = b
#                     print '%s at %d' % (aa_flag, new_i + 1)
                alignment_df = alignment_df.append(appender, ignore_index = True)
                idx_start = new_i + 1
                


            elif (i+1) == len(a_seq) and aa_flag_tracker == aa_flag:
                idx_end = new_i + 1
                if aa_flag_tracker != 'mutation' and aa_flag_tracker != 'insertion':
                    appender = {}
                    appender['id_a'] = a_seq_id
                    appender['id_b'] = b_seq_id
                    appender['type'] = aa_flag_tracker
                    appender['id_a_start'] = idx_start
                    appender['id_a_stop'] = idx_end
                    appender['count'] = idx_end - idx_start + 1
#                     print '%s from %d to %d with %d members' % (appender['type'], appender['start'], appender['stop'], appender['count'])
                    alignment_df = alignment_df.append(appender, ignore_index = True)

            elif (i+1) == len(a_seq) and aa_flag_tracker != aa_flag:
                idx_end = new_i + 1
                idx_start = new_i + 1
                if aa_flag_tracker != 'mutation' and aa_flag_tracker != 'insertion':
                    appender = {}
                    appender['id_a'] = a_seq_id
                    appender['id_b'] = b_seq_id
                    appender['type'] = aa_flag
                    appender['id_a_start'] = idx_start
                    appender['id_a_stop'] = idx_end
                    appender['count'] = idx_end - idx_start + 1
#                     print '%s from %d to %d with %d members' % (appender['type'], appender['start'], appender['stop'], appender['count'])
                    alignment_df = alignment_df.append(appender, ignore_index = True)
#             print new_i + 1, ':', a, b, 'last=%s, now=%s' % (aa_flag_tracker,aa_flag) 

            aa_flag_tracker = aa_flag
            
    return alignment_df

In [90]:
def get_alignment_allpos_df(alignment_file, a_seq_id=None, b_seq_id=None):
    alignments = list(AlignIO.parse(alignment_file, "emboss"))

    appender = defaultdict(dict)
    idx = 0
    for alignment in alignments:
    #         if not switch:
        if not a_seq_id:
            a_seq_id = list(alignment)[0].id
        a_seq = str(list(alignment)[0].seq)
        if not b_seq_id:
            b_seq_id = list(alignment)[1].id
        b_seq = str(list(alignment)[1].seq)

        a_idx = 1
        b_idx = 1

        for i, (a,b) in enumerate(zip(a_seq,b_seq)):
            if a == b and a != '-' and b != '-':
                aa_flag = 'match'
            if a != b and a == '-' and b != '-':
                aa_flag = 'insertion'
            if a != b and a != '-' and b == '-':
                aa_flag = 'deletion'
            if a != b and a != '-' and b == 'X':
                aa_flag = 'unresolved'
            if a != b and b != '-' and a == 'X':
                aa_flag = 'unresolved'
            elif a != b and a != '-' and b != '-':
                aa_flag = 'mutation'
                
            appender[idx]['id_a'] = a_seq_id
            appender[idx]['id_b'] = b_seq_id
            appender[idx]['type'] = aa_flag
            
            if aa_flag == 'match' or aa_flag == 'unresolved' or aa_flag == 'mutation':
                appender[idx]['id_a_aa'] = a
                appender[idx]['id_a_pos'] = a_idx
                appender[idx]['id_b_aa'] = b
                appender[idx]['id_b_pos'] = b_idx
                a_idx += 1
                b_idx += 1

            if aa_flag == 'deletion':
                appender[idx]['id_a_aa'] = a
                appender[idx]['id_a_pos'] = a_idx
                a_idx += 1

            if aa_flag == 'insertion':
                appender[idx]['id_b_aa'] = b
                appender[idx]['id_b_pos'] = b_idx
                b_idx += 1
            
            idx += 1

    alignment_df = pd.DataFrame.from_dict(appender, orient='index')
    alignment_df = alignment_df[['id_a', 'id_b', 'type', 'id_a_aa', 'id_a_pos', 'id_b_aa', 'id_b_pos']].fillna(value=np.nan)
    
#     alignment_df = alignment_df.join(alignment_df['id_a'].apply(lambda x: pd.Series(x.split('.')[1])))
#     alignment_df = alignment_df.rename(columns={0:'id_a_chain'})
    
#     alignment_df = alignment_df.join(alignment_df['id_b'].apply(lambda x: pd.Series(x.split('.')[1])))
#     alignment_df = alignment_df.rename(columns={0:'id_b_chain'})
    
    return alignment_df

***
## COBRA & SBML RELATED

In [59]:
def is_spontaneous(gene):
    spont = re.compile("[Ss](_|)0001")
    if spont.match(gene):
        return True
    else:
        return False

In [63]:
def true_num_genes(model):
    true_num = 0
    for gene in model.genes:
        if not is_spontaneous(gene.id):
            true_num += 1
    return true_num

In [65]:
def true_num_reactions(model):
    true_num = 0
    for rxn in model.reactions:
        genes = [x.id for x in rxn.genes]
        if len(genes) == 0:
            continue
        if len(genes) == 1 and is_spontaneous(genes[0]):
            continue
        else:
            true_num += 1
    return true_num

In [ ]:
def adj_num_reactions(model, missing_genes):
    adj_num = 0
    for rxn in model.reactions:
        genes = [x.id for x in rxn.genes]
        if len(genes) == 0:
            continue
        if len(genes) == 1 and (is_spontaneous(genes[0]) or genes[0] in missing_genes):
            continue
        else:
            adj_num += 1
    return adj_num  

In [9]:
def gene_name_to_id(full_model_xml):
    '''
    Input: the .xml filename of the 'full' model (in my case Recon2) which has the gene names and associated gene IDs listed as species
    Output: a dictionary with the following structure - {GeneName: GeneID, ...}
    '''

    full_model_sbml = reader.readSBML(full_model_xml)
    m = full_model_sbml.getModel()

    gene_dict = defaultdict(dict)

    # 'species' includes the genes
    for i in m.getListOfSpecies():
        entrez = i.getId()

        # some start with _NM
        if entrez.startswith('_NM'):
            name = i.getName()
            annotation = i.getAnnotation()
            gene_dict[name] = entrez

        # all other genes start with _
        elif entrez.startswith('_'):
            name = i.getName()
            annotation = i.getAnnotation()
            idz = entrez.split('_')
            idz.pop(0)

            # accounting for genes that are connected
            newidz = idz[0::3]
            namez = name.split(':')
            for x in range(len(newidz)):
                if namez[x] == 'null':
                    continue
                gene_dict[namez[x]] = newidz[x]

    return gene_dict

In [10]:
def full_to_reduced(full_dict, reduced_list_of_genes):
    '''
    Input:
        full_dict: the dictionary returned by function nameToId
        reduced_list_of_genes: the list of genes of your reduced model (in my case iAB-RBC)
    Output:
        gene_mapping: the gene mapping for your reduced model (a dictionary of dictionaries with form {GeneName: {'ENTREZ': GeneID, 'UNIPROT': UniprotAcc#}, ...})
        missing_in_full: genes that were not in the full model but in the reduced - stuff that needs to be looked at manually
    '''
    missing_in_full = []
    gene_mapping = {}

    for gene in reduced_list_of_genes:
        genename = gene.name.strip('|&').split('.')[0].upper()
        if genename in full_dict:
            gene_mapping[genename] = full_dict[genename]
        else:
            missing_in_full.append(genename)

    return gene_mapping, list(set(filter(None, missing_in_full)))

***
## MISCELLANEOUS

In [11]:
def write_pickle(filePath,d):
    f=open(filePath,'w')
    newData = pickle.dumps(d, 1)
    f.write(newData)
    f.close()

def read_pickle(filePath):
    f=open(filePath,'r')
    data = pickle.load(f)
    f.close()
    return data

***
## DEPRECATED

In [ ]:
# UNIPROT
# DEPRECATED: replaced by bioservices
# used for uniprot information: https://github.com/boscoh/uniprot
import uniprot

### Function: general_mapper
#### ANY ID & INPUT TYPE & OUTPUT TYPE -> MAPPED ID
This function uses the UniProt mapping tool to map any ID to another ID. You need to specify the ID types though.

In [10]:
def general_mapper(map_from, map_to, gene_id):
    '''
    Test out a mapping here: http://www.uniprot.org/mapping/
    Specify ID types(!) from here: http://www.uniprot.org/faq/28#id_mapping_examples
    reviewed=True for only reviewed UniProt entries
    '''
    
    pairs = uniprot.batch_uniprot_id_mapping_pairs(map_from, map_to, [gene_id])

    maps = []
    for idx in range(len(pairs)):
        maps.append(str(pairs[idx][1]))

    return maps

### Function: uniprot_mapper
#### ANY ID & INPUT TYPE -> UNIPROT ID

This function will use the UniProt mapping tool and return a list of UniProt IDs associated with another ID.

You need to know where your original ID comes from, and enter the code from http://www.uniprot.org/faq/28#id_mapping_examples

By default it will return all UniProt IDs, including unreviewed ones. Put reviewed=True if you want only reviewed ones

In [11]:
def uniprot_mapper(gene_id, id_type, reviewed=False):
    '''
    Test out a mapping here: http://www.uniprot.org/mapping/
    Specify ID type from here: http://www.uniprot.org/faq/28#id_mapping_examples
    reviewed=True for only reviewed UniProt entries
    '''
    
    pairs = uniprot.batch_uniprot_id_mapping_pairs(id_type, 'ACC', [gene_id])

    unis = []
    for idx in range(len(pairs)):
        unis.append(str(pairs[idx][1]))

    if reviewed == True:
        meta = uniprot.batch_uniprot_metadata(unis)
        for uni_id in meta.keys():
            if meta[uni_id]['is_reviewed'] == False:
                del meta[uni_id]
        return meta.keys()

    else:
        return unis

### Function: get_pdb_seq_and_rez
#### PDB ID -> DOWNLOAD/PARSE PDB & STRUCTURE SEQUENCE & RESOLUTION

In [27]:
def get_pdb_seq_and_rez(pdbid):
    '''
    This function downloads a PDB ID (biological assembly) into the working directory and also
    returns the sequence that is represented in the structure (what residues are visible).
    The sequence is a SeqRecord Biopython object.

    Input: pdbid - a 4 letter string
    Output: structure_seqs - a Seq object that represents what residues can be seen in the structure
            resolution - the crystal structure resolution
    '''

    try:
        # change to biomol=True if you want biological assemblies
        pdb, header = pr.parsePDB(pdbid, header=True, biomol=False, secondary=False)

        if 'NMR' in header['experiment'] or 'nmr' in header['experiment']:
            warnings.warn('******WARNING: NMR entry %s******' % pdbid)

        # if multiple biomols choose the first one (this could be improved)
        if type(pdb) == list:
            pdb = pdb[0]

    except IOError:
        return None, None
    # if there are no biological assemblies
    except ValueError:
        pdb, header = pr.parsePDB(pdbid, header=True, biomol=False, secondary=False)

    if 'resolution' not in header:
        resolution = None
    else:
        resolution = float(header['resolution'])

    ## if doing bio assembly NEED TO CONSIDER "SEGMENTS"!!
    if pdb.numSegments() > 0:
        structure_seqs = []
        for seg in pdb.iterSegments():
            sg = seg.getSegname()
            for chain in seg.iterChains():
                ch = chain.getChid()
                seq = SeqRecord(Seq(chain.getSequence(), IUPAC.protein), id=pdbid+'.'+sg+'.'+ch, description='PDB structure residues, biomol 1')
                structure_seqs.append(seq)
    else:
        structure_seqs = []
        for chain in pdb.iterChains():
            ch = chain.getChid()
            seq = SeqRecord(Seq(chain.getSequence(), IUPAC.protein), id=pdbid+'.'+ch, description='PDB structure residues')
            structure_seqs.append(seq)

    return structure_seqs, resolution

****

## NEW FUNCTIONS

### Function: get_org_from_pdb
#### PDB ID -> DOWNLOAD/PARSE PDB & FIND WHICH ORGANISM THE TEMPLATE IS DERIVED FROM

In [65]:
from bs4 import *
import urllib2
from urllib2 import Request

def get_pdb_info(pdbid):
    tmp_url='http://www.rcsb.org/pdb/rest/describeMol?structureId=%s'%pdbid
    req = Request(tmp_url)
    res = urllib2.urlopen(req)
    soup = BeautifulSoup(urllib2.urlopen(req))
    tmp_org = str(soup.contents[0].taxonomy.get("id"))
    return tmp_org
    
def get_org_from_pdb(pdb_file):
    pdb = get_pdb_info(pdb_file)
    uniprot_url = 'http://www.uniprot.org/taxonomy/%s'%pdb
    req = Request(uniprot_url)
    res = urllib2.urlopen(req)
    soup = BeautifulSoup(urllib2.urlopen(req))
    if 'unidentified' in str(soup.title.get_text()):
        tmp ='unidentified'
    else:
        tmp = soup.title.get_text().split()[0]+" "+soup.title.get_text().split()[1]
    return tmp.encode('utf8')

# Usage: organism = get_org_from_pdb(pdbid)

### Function: blast_pdb
#### seq -> dictionary of blast results from blasting entire pdb

In [ ]:
import xmltodict as xd

def blast_pdb(seq, evalue = 0.001):
    '''
    Returns the first BLASTp hit with a default evalue search param of 0.001. 
    returns a dict of..  %TODO: lots of stuff!
    up to u if u think its a good hit.
    '''
    print 'http://www.rcsb.org/pdb/rest/getBlastPDB1?sequence=%s&eCutOff=%s&matrix=BLOSUM62&outputFormat=XML' % (seq, evalue)
    req = urllib2.Request('http://www.rcsb.org/pdb/rest/getBlastPDB1?sequence=%s&eCutOff=%s&matrix=BLOSUM62&outputFormat=XML' % (seq, evalue))
    response = urllib2.urlopen(req)
    
    if response.getcode() == 200:
        info_dict = {}
        len_orig = len(seq)

        raw = sio.StringIO(response.read())
        doc = de_unicodeify(xd.parse(raw))
        
        if 'Iteration_hits' not in doc['BlastOutput']['BlastOutput_iterations']['Iteration'].keys():
            return None
        else:
            if isinstance(doc['BlastOutput']['BlastOutput_iterations']['Iteration']['Iteration_hits']['Hit'], list):
                best_hit = doc['BlastOutput']['BlastOutput_iterations']['Iteration']['Iteration_hits']['Hit'][0]
            else:
                best_hit = doc['BlastOutput']['BlastOutput_iterations']['Iteration']['Iteration_hits']['Hit']
            hit_pdb = best_hit['Hit_def'].split('|')[0].split(':')[0]
            hit_pdb_chain = best_hit['Hit_def'].split('|')[0].split(':')[2]

            hit_dict = de_unicodeify(best_hit['Hit_hsps']['Hsp'])

            if isinstance(hit_dict, list):
                hit_dict = hit_dict[0]

            info_dict['input_seq'] = seq
            info_dict['input_seq_len'] = len_orig
            info_dict['hit_pdb'] = hit_pdb
            info_dict['hit_pdb_chain'] = hit_pdb_chain
            info_dict['hit_seq'] = hit_dict['Hsp_hseq']
            info_dict['hit_num_ident'] = hit_dict['Hsp_identity']
            info_dict['hit_percent_ident'] = float(hit_dict['Hsp_identity'])/float(len_orig)
            info_dict['hit_evalue'] = hit_dict['Hsp_evalue']
            info_dict['hit_score'] = hit_dict['Hsp_score']
    
            return info_dict
        
    else:
        return None